# Horizontal-to-Vertical Spectral Ratio (HVSR) Measurements from Microtremor Recordings (mHVSR)

## License Information

This file is part of _hvsrpy_, a Python package for horizontal-to-vertical spectral ratio processing.

    Copyright (C) 2019-2024 Joseph P. Vantassel (joseph.p.vantassel@gmail.com)

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <https: //www.gnu.org/licenses/>.
    
## About _hvsrpy_

_hvsrpy_ is an open-source Python package for performing horizontal-to-vertical spectral ratio (HVSR) processing of microtremor and earthquake recordings. _hvsrpy_ was developed by [Joseph P. Vantassel](https://www.jpvantassel.com/) with contributions from Dana M. Brannon under the supervision of Professor Brady R. Cox at The University of Texas at Austin. _hvsrpy_ continues to be developed and maintained by [Joseph P. Vantassel and his research group at Virginia Tech](https://geoimaging-research.org/).

## Citation

If you use _hvsrpy_ in your research or consulting, we ask you please cite the following:

>Joseph Vantassel. (2020). jpvantassel/hvsrpy: latest (Concept). Zenodo.
[http://doi.org/10.5281/zenodo.3666956](http://doi.org/10.5281/zenodo.3666956)

_Note: For software, version specific citations should be preferred to
general concept citations, such as that listed above. To generate a version
specific citation for hvsrpy, please use the citation tool on the hvsrpy
[archive](http://doi.org/10.5281/zenodo.3666956)._

## About this notebook

This notebook demonstrates traditional mHVSR processing using _hvsrpy_.

The lognormal statistics implemented in _hvsrpy_ were developed by Cox et al. (2020).
If you use this feature, please also cite Cox et al. (2020) to recognize their original work.

> Cox, B. R., Cheng, T., Vantassel, J. P., & Manuel, L. (2020). "A statistical
> representation and frequency-domain window-rejection algorithm for single-station
> HVSR measurements. Geophysical Journal International, 221(3), 2170–2183.
> https://doi.org/10.1093/gji/ggaa119

The microtremor data presented are those by Cox and Vantassel (2018).
If you use this data, please also cite Cox and Vantassel (2018).

>Cox, B.R. and Vantassel, J.P. (2018). Dynamic Characterization of Wellington, New Zealand. DesignSafe-CI [publisher]. https://doi.org/10.17603/DS24M6J

## Getting Started

1. Install _hvsrpy_, with `pip install hvsrpy`. More information on _pip_ can be found [here](https://jpvantassel.github.io/python3-course/#/intro/pip). __(~3 minutes)__
2. Try analyzing the provided examples. __(~10 minutes)__
3. Try your own files by changing the file names below. __(~15 minutes)__

Happy Processing!

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
from collections import defaultdict
import hvsrpy
from hvsrpy import sesame
from io import StringIO
import contextlib

def safe_print(s):
    try:
        print(s)
    except UnicodeEncodeError:
        print(s.encode('utf-8', errors='replace').decode('utf-8'))


plt.style.use(hvsrpy.HVSRPY_MPL_STYLE)

## Input Data

In [22]:
# ✅ 데이터 폴더 지정
data_dir = pathlib.Path("C:/SOLODATA/zonghap/3month/3월부터얻은데이터목록")
sac_files = sorted(data_dir.glob("*.sac"))

# ✅ Z/N/E 그룹화
grouped_files = defaultdict(dict)
for file in sac_files:
    parts = file.stem.split(".")
    if len(parts) < 9:
        continue
    base_id = ".".join(parts[:-1])
    comp = parts[-1]
    grouped_files[base_id][comp] = str(file)

fname_sets = []
for base, comps in grouped_files.items():
    if all(k in comps for k in ["E", "N", "Z"]):
        fname_sets.append([comps["Z"], comps["N"], comps["E"]])

# ✅ 저장 폴더
save_root = pathlib.Path("C:/SOLODATA/zonghap/hvsr_output")
output_dir = save_root / data_dir.name
output_dir.mkdir(parents=True, exist_ok=True)

## Preprocessing Settings

In [23]:
preprocessing_settings = hvsrpy.settings.HvsrPreProcessingSettings()
preprocessing_settings.detrend = "linear"
preprocessing_settings.window_length_in_seconds = 30
preprocessing_settings.orient_to_degrees_from_north = 0.0
preprocessing_settings.filter_corner_frequencies_in_hz = (1, 10)
preprocessing_settings.ignore_dissimilar_time_step_warning = False

print("Preprocessing Summary")
print("-"*60)
preprocessing_settings.psummary()

Preprocessing Summary
------------------------------------------------------------
hvsrpy_version                           : 2.0.0
orient_to_degrees_from_north             : 0.0
filter_corner_frequencies_in_hz          : (1, 10)
window_length_in_seconds                 : 30
detrend                                  : linear
preprocessing_method                     : hvsr


## Processing Settings

In [24]:
processing_settings = hvsrpy.settings.HvsrTraditionalProcessingSettings()
processing_settings.window_type_and_width = ("tukey", 0.2)
processing_settings.smoothing=dict(operator="konno_and_ohmachi",
                                   bandwidth=40,
                                   center_frequencies_in_hz=np.geomspace(0.2, 50, 200))
processing_settings.method_to_combine_horizontals = "geometric_mean"
processing_settings.handle_dissimilar_time_steps_by = "frequency_domain_resampling"

print("Processing Summary")
print("-"*60)
processing_settings.psummary()

Processing Summary
------------------------------------------------------------
hvsrpy_version                           : 2.0.0
window_type_and_width                    : ('tukey', 0.2)
smoothing                                :
     operator                            : konno_and_ohmachi
     bandwidth                           : 40
     center_frequencies_in_hz            : [0.2, 0.205626908165 ... 1767550226364, 50.0]
fft_settings                             : None
handle_dissimilar_time_steps_by          : frequency_domain_resampling
processing_method                        : traditional
method_to_combine_horizontals            : geometric_mean


## Calculate HVSR

In [25]:
# ✅ 루프 시작
for fname_set in fname_sets:
    base_name = pathlib.Path(fname_set[0]).stem.replace(".Z", "")
    print(f"\n▶ Processing: {base_name}")

    try:
        srecords = hvsrpy.read([fname_set])
        srecords = hvsrpy.preprocess(srecords, preprocessing_settings)
        hvsr = hvsrpy.process(srecords, processing_settings)
        search_range_in_hz = (1, 10)
        verbose = 1
        hvsr.update_peaks_bounded(search_range_in_hz=search_range_in_hz)
        n = 2
        search_range_in_hz = (1, 10)
        _ = hvsrpy.frequency_domain_window_rejection(hvsr, n=n, search_range_in_hz=search_range_in_hz)

        # ✅ 요약 표 저장 (CSV)
        rounded_fn= round(fn,3)
        summary_df = pd.DataFrame({
            "Statistic": [
                "Resonant Site Frequency, fn (Hz)",
                "Resonant Site Period, Tn (s)",
                "Resonance Amplitude, An",
                "Peak of the mean curve"
            ],
            "Exponentiated Lognormal Median (units)": [fn, Tn, An,rounded_fn],
            "Lognormal Standard Deviation (log units)": [fn_std_log, fn_std_log, An_std_log,""],
            "-1 Lognormal Standard Deviation (units)": [fn_lo, Tn_lo, An_lo,""],
            "+1 Lognormal Standard Deviation (units)": [fn_hi, Tn_hi, An_hi,""]
        })

        csv_path = output_dir / f"{base_name}_summary.csv"
        with open(csv_path, "w", encoding="utf-8", newline="") as f:
            summary_df.to_csv(f, index=False)
            f.write("\n")
            f.write(f"The peak of the mean curve is at {fn:.3f} Hz with amplitude {An:.3f}.\n")

        # ✅ 그래프 저장 (PNG)
        fig, ax = hvsrpy.plot_single_panel_hvsr_curves(hvsr)

        # ✅ 기타 설정 (기존 코드 유지)
        ax.get_legend().remove()
        ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))
        fig.savefig(output_dir / f"{base_name}_hvsr.png", bbox_inches="tight")
        plt.close(fig)

    except Exception as e:
        print(f"⚠️ 오류 발생: {base_name} - {e}")



▶ Processing: 453015908.0001.2025.03.23.03.34.18.000


,Exponentitated Lognormal Median (units),Lognormal Standard Deviation (log units),-1 Lognormal Standard Deviation (units),+1 Lognormal Standard Deviation (units)
"Resonant Site Frequency, fn (Hz)",1.257,0.124,1.111,1.423
"Resonant Site Period, Tn (s)",0.795,0.124,0.900,0.703
"Resonance Amplitude, An",13.995,0.467,8.774,22.323



▶ Processing: 453015908.0002.2025.03.23.04.40.34.000


,Exponentitated Lognormal Median (units),Lognormal Standard Deviation (log units),-1 Lognormal Standard Deviation (units),+1 Lognormal Standard Deviation (units)
"Resonant Site Frequency, fn (Hz)",1.868,0.595,1.030,3.387
"Resonant Site Period, Tn (s)",0.535,0.595,0.971,0.295
"Resonance Amplitude, An",10.931,0.480,6.763,17.667



▶ Processing: 453015908.0003.2025.03.23.05.43.22.000


...

,Exponentitated Lognormal Median (units),Lognormal Standard Deviation (log units),-1 Lognormal Standard Deviation (units),+1 Lognormal Standard Deviation (units)
"Resonant Site Frequency, fn (Hz)",1.182,0.105,1.064,1.313
"Resonant Site Period, Tn (s)",0.846,0.105,0.940,0.762
"Resonance Amplitude, An",5.796,0.466,3.638,9.236



▶ Processing: 453018809.0012.2025.04.08.23.10.34.000


,Exponentitated Lognormal Median (units),Lognormal Standard Deviation (log units),-1 Lognormal Standard Deviation (units),+1 Lognormal Standard Deviation (units)
"Resonant Site Frequency, fn (Hz)",1.213,0.112,1.084,1.357
"Resonant Site Period, Tn (s)",0.824,0.112,0.922,0.737
"Resonance Amplitude, An",5.677,0.512,3.401,9.476



▶ Processing: 453018809.0013.2025.04.09.00.00.00.000

▶ Processing: 453018809.0014.2025.04.09.00.21.46.000


,Exponentitated Lognormal Median (units),Lognormal Standard Deviation (log units),-1 Lognormal Standard Deviation (units),+1 Lognormal Standard Deviation (units)
"Resonant Site Frequency, fn (Hz)",1.292,0.179,1.080,1.545
"Resonant Site Period, Tn (s)",0.774,0.179,0.926,0.647
"Resonance Amplitude, An",6.197,0.565,3.523,10.900
